In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime


In [15]:
def clean_text(text: str) -> str:
    text = text.strip(' \n\t')
    text = text.replace('\t','')
    text_list = text.split('\n')
    new_list = []
    for t in text_list:
        t = t.strip()
        if t != '' and not t.isspace():
            new_list.append(t)
    return '\n'.join(new_list)


In [16]:
URL = 'https://fundusze.ngo.pl/aktualne?when=ever'
r = requests.get(URL)
soup = BeautifulSoup(r.content)

In [17]:
pages = int(soup.find('div', class_='cms').text.split(' ')[1])//20+1
print(f"Jest {pages} stron.")
block_str= {5: ('name', 'descr', 'params', 'tags', 'author'),
            4: ('name', 'params', 'tags', 'author')}
params_str = ('obszar','nabor_do','fundusz','budzet','procent','kwota')

all_grants_list = []

for i in range(1,pages+1): # pages+1
    psoup = BeautifulSoup(requests.get(URL+f'&page={i}').content)
    boxes = psoup.find_all('div', class_='mb5')
    grants = None
    for b in boxes:
        if b.text.strip().startswith('Znalezione konkursy'):
            grants = b
    grants = grants.find_all('div', class_='grow-1')
    print(f"Na stronie {i} znaleziono {len(grants)} grantów.")
    for n,g in enumerate(grants):
        try:
            gdesc = g.find('div')
            gblocks = gdesc.find_all('div', recursive=False)
            if len(gblocks) not in (4,5):
                print(f"Zła liczba bloków: {len(gblocks)}")
                continue
            grant_dict = {}
            for i,name in enumerate(block_str[len(gblocks)]):
                grant_dict[name] = clean_text(gblocks[i].text)
                if name == 'name':
                    grant_dict['url'] = gblocks[i].find('a')['href']
            grant_dict['tags'] = grant_dict['tags'].replace('\n','')
            gparams = gblocks[-3].find_all('div',class_='lh-title')
            if len(gparams) < 6:
                print(f'not enough params for {grant_dict["name"]}')
            for i,name in enumerate(params_str):
                grant_dict[name] = clean_text(gparams[i].text).replace('\n',' ').strip('\n ')
            print(grant_dict)
            all_grants_list.append(grant_dict)
        except IndexError as e:
            print(f"{e}")
            continue

df = pd.DataFrame(all_grants_list)
df

            

Jest 26 stron.
Na stronie 1 znaleziono 21 grantów.
Zła liczba bloków: 0
{'name': 'Pedagodzy ulicy - otwarty konkurs ofert na wsparcie realizacji zadań publicznych w zakresie wspierania rodziny...', 'url': 'https://fundusze.ngo.pl/443483-pedagodzy-ulicy-otwarty-konkurs-ofert-na-wsparcie-realizacji-zadan-publicznych-w-zakresie-wspierania-rodziny-i-systemu-pieczy-zastepczej-w-2024-roku.html', 'descr': 'Pedagodzy ulicy - otwarty konkurs ofert na wsparcie realizacji zadań publicznych w zakresie wspierania rodziny i systemu pieczy zastępczej w 2024 roku.\nUrząd Miejski w Olecku, Pełnomocnik ds. Przeciwdziałania Uzależnieniom ogłasza otwarty konkurs ofert na realizację zadania publicznego pod nazwą: pedagodzy ulicy - otwarty konkurs ofert na wsparcie realizacji zadań publicznych w\xa0zakresie wspierania rodziny i systemu pieczy zastępczej w\xa02024\xa0roku.', 'params': 'Olecko\nNabór do\n04.03.2024\nMiejsko-gminny\nŁączny budżet 46,2 tys. PLN\nProcent dofinansowania nieznany\nDotacje do 46,2 

,name,url,descr,params,tags,author,obszar,nabor_do,fundusz,budzet,procent,kwota
0,Pedagodzy ulicy - otwarty konkurs ofert na wsp...,https://fundusze.ngo.pl/443483-pedagodzy-ulicy...,Pedagodzy ulicy - otwarty konkurs ofert na wsp...,Olecko\nNabór do\n04.03.2024\nMiejsko-gminny\n...,"Edukacja i wychowanie, Polityka społeczna",Pełnomocnik ds. Przeciwdziałania Uzależnieniom,Olecko,Nabór do 04.03.2024,Miejsko-gminny,"Łączny budżet 46,2 tys. PLN",Procent dofinansowania nieznany,"Dotacje do 46,2 tys."
1,Świetlice - otwarty konkurs ofert na wsparcie ...,https://fundusze.ngo.pl/443479-swietlice-otwar...,Świetlice - otwarty konkurs ofert na wsparcie ...,Olecko\nNabór do\n04.03.2024\nMiejsko-gminny\n...,"Edukacja i wychowanie, Polityka społeczna",Pełnomocnik ds. Przeciwdziałania Uzależnieniom,Olecko,Nabór do 04.03.2024,Miejsko-gminny,Łączny budżet 144 tys. PLN,Procent dofinansowania nieznany,Dotacje do 144 tys.
2,Klub młodzieżowy - otwarty konkurs ofert na ws...,https://fundusze.ngo.pl/443474-klub-mlodziezow...,Klub młodzieżowy - otwarty konkurs ofert na ws...,Olecko\nNabór do\n04.03.2024\nMiejsko-gminny\n...,"Edukacja i wychowanie, Polityka społeczna",Pełnomocnik ds. Przeciwdziałania Uzależnieniom,Olecko,Nabór do 04.03.2024,Miejsko-gminny,"Łączny budżet 47,3 tys. PLN",Procent dofinansowania nieznany,"Dotacje do 47,3 tys."
3,„Zdrowie publiczne” w 2024 r. (Szkoła dla Rodz...,https://fundusze.ngo.pl/443615-zdrowie-publicz...,NaN,Skawina\nNabór do\n29.02.2024\nMiejsko-gminny\...,"Polityka społeczna, Ochrona zdrowia","Wydział Promocji, Sportu i Współpracy",Skawina,Nabór do 29.02.2024,Miejsko-gminny,"Łączny budżet 27,2 tys. PLN",Procent dofinansowania nieznany,Kwota dofinansowania nieznana
4,Nabór ofert w trybie art. 19a w zakresie promo...,https://fundusze.ngo.pl/443358-nabor-ofert-w-t...,NaN,wrocławski\nNabór do\n16.02.2024\nPowiatowy\nŁ...,Ochrona zdrowia,Wydział Promocji i Wsparcia Społecznego,wrocławski,Nabór do 16.02.2024,Powiatowy,Łączny budżet 30 tys. PLN,Procent dofinansowania nieznany,Dotacje do 10 tys.
5,PIECZA ZASTĘPCZA - Zorganizowanie specjalistyc...,https://fundusze.ngo.pl/443434-piecza-zastepcz...,NaN,Ochota\nNabór do\n29.02.2024\nMarszałkowski\nŁ...,"Edukacja i wychowanie, Polityka społeczna",Wydział Wspierania Rodziny i Systemu Pieczy Za...,Ochota,Nabór do 29.02.2024,Marszałkowski,"Łączny budżet 1,92 mln PLN",Procent dofinansowania nieznany,Kwota dofinansowania nieznana
6,PIECZA ZASTĘPCZA - Zorganizowanie specjalistyc...,https://fundusze.ngo.pl/443412-piecza-zastepcz...,NaN,Ochota\nNabór do\n29.02.2024\nMarszałkowski\nŁ...,Polityka społeczna,Wydział Wspierania Rodziny i Systemu Pieczy Za...,Ochota,Nabór do 29.02.2024,Marszałkowski,"Łączny budżet 1,62 mln PLN",Procent dofinansowania nieznany,Kwota dofinansowania nieznana
7,Konkurs grantowy Ambasady USA: American Spaces...,https://fundusze.ngo.pl/443586-konkurs-grantow...,NaN,Cała Polska\nNabór do\n04.04.2024\nZagraniczny...,"Aktywność obywatelska, Prawa człowieka, Działa...","Ambasada USA w Warszawie, Konsulat Generalny U...",Cała Polska,Nabór do 04.04.2024,Zagraniczny,Łączny budżet 125 tys. USD,Procent dofinansowania nieznany,Dotacje od 15 tys. do 50 tys.
8,Międzynarodowy Fundusz Wyszehradzki: Nowy sezo...,https://fundusze.ngo.pl/443579-miedzynarodowy-...,NaN,Cała Polska\nNabór do\n01.03.2024\nZagraniczny...,"Ukraina, Aktywność obywatelska, Działalność mi...",Międzynarodowy Fundusz Wyszehradzki,Cała Polska,Nabór do 01.03.2024,Zagraniczny,Łączny budżet nieznany,Procent dofinansowania nieznany,Dotacje do 10 tys.
9,Ponadnarodowy konkurs dotyczący łagodzenia spo...,https://fundusze.ngo.pl/443602-ponadnarodowy-k...,NaN,Cała Polska\nNabór do\n24.04.2024\nUnijny\nŁąc...,"Ukraina, Unia Europejska, Polityka społeczna",Europejskie Centrum Kompetencji,Cała Polska,Nabór do 24.04.2024,Unijny,Łączny budżet 22 mln EUR,Do 80% dofinansowania,Dotacje od 200 tys. do 700 tys.


In [12]:
df = df.drop('params', axis=1)

In [13]:
now = datetime.now()
ts = now.strftime('%y%m%d%H%M')
df.to_csv(f"grants_{ts}_{len(df)}.csv")